In [21]:
from datetime import datetime
import numpy as np 
import pandas as pd 
import glob 
import random 
import os 

# kmjng
base_folder =  '/home/user/miniconda3/envs/gpu_conda/src/data/ETRI_lifelog_dataset'

# # kmjng-nb
# base_folder = 'C:/Users/minjeong/Documents/my_env/env1/src/data/ETRI_lifelog_dataset'

folder = '/ch2025_data_items'

data_dir = base_folder + folder 


# Parquet 파일 전체 경로 리스트 
parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet')) 

# 파일 이름을 키로, DataFrame을 값으로 저장할 딕셔너리 
lifelog_data = {} 

# 파일별로 읽기 
for file_path in parquet_files: 
    name = os.path.basename(file_path).replace('.parquet', '').replace('ch2025_', '') 
    lifelog_data[name] = pd.read_parquet(file_path) 
    print(f"✅ Loaded: {name}, shape = {lifelog_data[name].shape}") 

# 딕셔너리에 있는 모든 항목을 독립적인 변수로 할당 
for key, df in lifelog_data.items(): 
    globals()[f"{key}_df"] = df 

# 메트릭스 파일 읽기
metrics_train = pd.read_csv(base_folder + '/ch2025_metrics_train.csv')
sample_submission = pd.read_csv(base_folder+'/ch2025_submission_sample.csv')

# ✅ 기준 쌍 (subject_id, lifelog_date) 
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']) 
test_keys = set(zip(sample_submission['subject_id'], sample_submission['lifelog_date'].dt.date)) 

# --------------------------- # 
# ✅ DataFrame 별 timestamp 컬럼 수동 지정 
dataframes = { 
    'mACStatus': (mACStatus_df, 'timestamp'), 
    'mActivity': (mActivity_df, 'timestamp'), 
    'mAmbience': (mAmbience_df, 'timestamp'), 
    'mBle': (mBle_df, 'timestamp'), 
    'mGps': (mGps_df, 'timestamp'), 
    'mLight': (mLight_df, 'timestamp'), 
    'mScreenStatus': (mScreenStatus_df, 'timestamp'), 
    'mUsageStats': (mUsageStats_df, 'timestamp'), 
    'mWifi': (mWifi_df, 'timestamp'), 
    'wHr': (wHr_df, 'timestamp'), 
    'wLight': (wLight_df, 'timestamp'), 
    'wPedo': (wPedo_df, 'timestamp'), 
} 
# ✅ 분리 함수 
def split_test_train(df, subject_col='subject_id', timestamp_col='timestamp'): 
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce') 
    df = df.dropna(subset=[timestamp_col]) 
    df['date_only'] = df[timestamp_col].dt.date 
    df['key'] = list(zip(df[subject_col], df['date_only'])) 
    test_df = df[df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    train_df = df[~df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    return test_df, train_df 


# ✅ 결과 저장 
for name, (df, ts_col) in dataframes.items(): 
    print(f"⏳ {name} 분리 중...") 
    test_df, train_df = split_test_train(df.copy(), subject_col='subject_id', timestamp_col=ts_col) 
    globals()[f"{name}_test"] = test_df 
    globals()[f"{name}_train"] = train_df 
    print(f"✅ {name}_test → {test_df.shape}, {name}_train → {train_df.shape}") 

✅ Loaded: wHr, shape = (382918, 3)
✅ Loaded: mGps, shape = (800611, 3)
✅ Loaded: mScreenStatus, shape = (939653, 3)
✅ Loaded: mACStatus, shape = (939896, 3)
✅ Loaded: mActivity, shape = (961062, 3)
✅ Loaded: mBle, shape = (21830, 3)
✅ Loaded: mWifi, shape = (76336, 3)
✅ Loaded: mLight, shape = (96258, 3)
✅ Loaded: mAmbience, shape = (476577, 3)
✅ Loaded: wLight, shape = (633741, 3)
✅ Loaded: wPedo, shape = (748100, 9)
✅ Loaded: mUsageStats, shape = (45197, 3)
⏳ mACStatus 분리 중...
✅ mACStatus_test → (335849, 3), mACStatus_train → (604047, 3)
⏳ mActivity 분리 중...
✅ mActivity_test → (343579, 3), mActivity_train → (617483, 3)
⏳ mAmbience 분리 중...
✅ mAmbience_test → (170453, 3), mAmbience_train → (306124, 3)
⏳ mBle 분리 중...
✅ mBle_test → (8140, 3), mBle_train → (13690, 3)
⏳ mGps 분리 중...
✅ mGps_test → (287386, 3), mGps_train → (513225, 3)
⏳ mLight 분리 중...
✅ mLight_test → (34439, 3), mLight_train → (61819, 3)
⏳ mScreenStatus 분리 중...
✅ mScreenStatus_test → (336160, 3), mScreenStatus_train → (60349

In [ ]:

def get_time_block(hour): 
    if 0 <= hour < 6: 
        return 'early_morning' 
    elif 6 <= hour < 12: 
        return 'morning' 
    elif 12 <= hour < 18: 
        return 'afternoon' 
    else: 
        return 'evening' 

def process_wHr_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            hr_all = [] 
            for row in block_group['heart_rate']:  # 리스크 값들 병합
                parsed = ast.literal_eval(row) if isinstance(row, str) else row 
                hr_all.extend([int(h) for h in parsed if h is not None]) 
            if not hr_all: 
                continue 
            above_100 = [hr for hr in hr_all if hr > 100] 
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all) 
            block_stats[f'hr_{block}_std'] = np.std(hr_all) 
            block_stats[f'hr_{block}_max'] = np.max(hr_all) 
            block_stats[f'hr_{block}_min'] = np.min(hr_all) 
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

wHr_df2 = process_wHr_by_timeblock(wHr_df) 
wHr_df2.head(2)

,subject_id,date,hr_afternoon_mean,hr_afternoon_std,hr_afternoon_max,hr_afternoon_min,hr_afternoon_above_100_ratio,hr_evening_mean,hr_evening_std,hr_evening_max,...,hr_early_morning_mean,hr_early_morning_std,hr_early_morning_max,hr_early_morning_min,hr_early_morning_above_100_ratio,hr_morning_mean,hr_morning_std,hr_morning_max,hr_morning_min,hr_morning_above_100_ratio
0,id01,2024-06-26,80.533300,12.636566,142.0,59.0,0.077268,82.476838,10.293221,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id01,2024-06-27,87.405165,12.296994,125.0,63.0,0.179430,83.526477,10.967673,122.0,...,86.170431,11.875367,113.0,66.0,0.160164,72.343956,10.333083,130.0,53.0,0.023445


* 기존 파생변수

In [34]:
wHr_df2.columns

Index(['subject_id', 'date', 'hr_afternoon_mean', 'hr_afternoon_std',
       'hr_afternoon_max', 'hr_afternoon_min', 'hr_afternoon_above_100_ratio',
       'hr_evening_mean', 'hr_evening_std', 'hr_evening_max', 'hr_evening_min',
       'hr_evening_above_100_ratio', 'hr_early_morning_mean',
       'hr_early_morning_std', 'hr_early_morning_max', 'hr_early_morning_min',
       'hr_early_morning_above_100_ratio', 'hr_morning_mean', 'hr_morning_std',
       'hr_morning_max', 'hr_morning_min', 'hr_morning_above_100_ratio'],
      dtype='object')

(*) 해당 날짜에 0~6시 사이 데이터가 없으면 early_morning 블록 자체가 존재하지 않음  
> id01의 2024-06-26에 early_morning 데이터가 없다면  
> hr_early_morning_mean, hr_early_morning_std 등의 키가 딕셔너리에 없음  

* 파생변수에서 NaN이 50% 이상인 시간대는 아예 제외 (피처 수 줄이기)

In [35]:
import pandas as pd
import numpy as np

def analyze_missing_by_timeblock(df, threshold=50):
    """
    시간대별 결측률 분석하고 드롭할 피처 결정
    threshold: 결측률 임계값 (%)
    """
    time_blocks = ['early_morning', 'morning', 'afternoon', 'evening']
    drop_features = []
    keep_features = []
    
    print("=" * 60)
    print("시간대별 결측률 분석")
    print("=" * 60)
    
    for block in time_blocks:
        # 해당 시간대의 모든 피처들
        block_features = [col for col in df.columns if f'hr_{block}_' in col]
        
        if not block_features:
            continue
            
        # 결측률 계산 (mean 컬럼 기준)
        mean_col = f'hr_{block}_mean'
        if mean_col in df.columns:
            missing_count = df[mean_col].isnull().sum()
            missing_pct = missing_count / len(df) * 100
            
            print(f"\n📊 {block.upper()}")
            print(f"   결측 개수: {missing_count}/{len(df)}")
            print(f"   결측률: {missing_pct:.1f}%")
            print(f"   관련 피처: {len(block_features)}개")
            
            if missing_pct > threshold:
                print(f"   ❌ 결측률이 {threshold}% 초과 -> 드롭 대상")
                drop_features.extend(block_features)
                print('  드롭할 피쳐: ', drop_features)
            else:
                print(f"   ✅ 결측률이 {threshold}% 이하 -> 유지")
                keep_features.extend(block_features)
    
    return drop_features, keep_features

# 실행
drop_features, keep_features = analyze_missing_by_timeblock(wHr_df2, threshold=50)

시간대별 결측률 분석

📊 EARLY_MORNING
   결측 개수: 411/636
   결측률: 64.6%
   관련 피처: 5개
   ❌ 결측률이 50% 초과 -> 드롭 대상
  드롭할 피쳐:  ['hr_early_morning_mean', 'hr_early_morning_std', 'hr_early_morning_max', 'hr_early_morning_min', 'hr_early_morning_above_100_ratio']

📊 MORNING
   결측 개수: 94/636
   결측률: 14.8%
   관련 피처: 5개
   ✅ 결측률이 50% 이하 -> 유지

📊 AFTERNOON
   결측 개수: 39/636
   결측률: 6.1%
   관련 피처: 5개
   ✅ 결측률이 50% 이하 -> 유지

📊 EVENING
   결측 개수: 103/636
   결측률: 16.2%
   관련 피처: 5개
   ✅ 결측률이 50% 이하 -> 유지


### 피쳐 드롭

In [ ]:
def drop_high_missing_features(df, drop_features):
    """
    높은 결측률 피처들 드롭
    """
    print("\n" + "=" * 60)
    print("피처 드롭 실행")
    print("=" * 60)
    
    if not drop_features:
        print("드롭할 피처가 없습니다.")
        return df
    
    print(f"드롭할 피처 수: {len(drop_features)}")
    print("드롭할 피처들:")
    for feature in drop_features:
        print(f"  - {feature}")
    
    df_cleaned = df.drop(columns=drop_features)
    
    print(f"\n변경사항:")
    print(f"  이전 피처 수: {df.shape[1]}")
    print(f"  이후 피처 수: {df_cleaned.shape[1]}")
    print(f"  제거된 피처 수: {len(drop_features)}")
    
    return df_cleaned

# 피처 드롭 실행
wHr_df2 = drop_high_missing_features(wHr_df2, drop_features)




피처 드롭 실행
드롭할 피처 수: 5
드롭할 피처들:
  - hr_early_morning_mean
  - hr_early_morning_std
  - hr_early_morning_max
  - hr_early_morning_min
  - hr_early_morning_above_100_ratio

변경사항:
  이전 피처 수: 22
  이후 피처 수: 17
  제거된 피처 수: 5


### 원본데이터 확인

In [ ]:
wHr_df.head(2) # heart_rate 길이가 제각각 다름 ! 

,subject_id,timestamp,heart_rate
0,id01,2024-06-26 12:23:00,"[134, 134, 135, 133, 134, 135, 134, 135, 134, ..."
1,id01,2024-06-26 12:24:00,"[123, 122, 121, 120, 121, 121, 120, 118, 119, ..."


 수면의질 관련 특화 특성  

심박수 구간별 비율:  

서맥 (<60), 정상 (60-100), 빈맥 (>100) 비율  


안정성 지표: 1 / (1 + 표준편차)  
일주기 리듬: 시간대별 심박수 패턴  

### 파생변수 추가   
> 밤-낮 심박수 차이, 정상심박수 비율, 심박수 안정성

In [45]:
import ast 

def get_time_block_no_early(hour): 
    if 6 <= hour < 12: 
        return 'morning' 
    elif 12 <= hour < 18: 
        return 'afternoon' 
    elif 18 <= hour < 24:
        return 'evening'
    else:
        return None

def process_wHr_with_features_only(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block_no_early) 
    
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        
        # 전체 일일 심박수 데이터 수집
        all_day_hr = []
        day_hr = []    # 낮 (6-18시): morning + afternoon
        night_hr = []  # 밤 (18-24시): evening만
        
        valid_group = group[group['block'].notna()]
        
        for block, block_group in valid_group.groupby('block'): 
            hr_all = [] 
            for row in block_group['heart_rate']: 
                parsed = ast.literal_eval(row) if isinstance(row, str) else row 
                hr_all.extend([int(h) for h in parsed if h is not None]) 
            
            if not hr_all: 
                continue 
            
            all_day_hr.extend(hr_all)
            
            # 낮/밤 분류만 하고 시간대별 통계는 제거
            if block in ['morning', 'afternoon']:
                day_hr.extend(hr_all)
            elif block == 'evening':
                night_hr.extend(hr_all)
        
        # 파생변수 3개만
        if day_hr and night_hr:
            block_stats['day_night_hr_diff'] = np.mean(day_hr) - np.mean(night_hr)
        else:
            block_stats['day_night_hr_diff'] = np.nan
        
        if all_day_hr:
            normal_hr = [hr for hr in all_day_hr if 60 <= hr <= 100]
            block_stats['normal_hr_ratio'] = len(normal_hr) / len(all_day_hr)
        else:
            block_stats['normal_hr_ratio'] = np.nan
        
        if all_day_hr and len(all_day_hr) > 1:
            hr_mean = np.mean(all_day_hr)
            hr_std = np.std(all_day_hr)
            block_stats['hr_stability'] = hr_std / hr_mean if hr_mean > 0 else np.nan
        else:
            block_stats['hr_stability'] = np.nan
            
        results.append(block_stats) 
    
    return pd.DataFrame(results) 

# 실행
wHr_df3 = process_wHr_with_features_only(wHr_df)
print(wHr_df3.columns.tolist())

['subject_id', 'date', 'day_night_hr_diff', 'normal_hr_ratio', 'hr_stability']


In [46]:
wHr_df3.head(5)

,subject_id,date,day_night_hr_diff,normal_hr_ratio,hr_stability
0,id01,2024-06-26,-1.943538,0.930558,0.146119
1,id01,2024-06-27,-7.571946,0.925296,0.158047
2,id01,2024-06-28,3.314891,0.922346,0.156781
3,id01,2024-06-29,-40.727273,0.714286,0.211625
4,id01,2024-06-30,NaN,0.864973,0.113833


In [49]:
nan_ratio = wHr_df3.isnull().sum() / len(wHr_df3) * 100
print("컬럼별 NaN 비율 (%):")
print(nan_ratio)

컬럼별 NaN 비율 (%):
subject_id            0.000000
date                  0.000000
day_night_hr_diff    18.867925
normal_hr_ratio       0.786164
hr_stability          1.729560
dtype: float64


### merge

In [ ]:
wHr_df3 = pd.merge(wHr_df2, wHr_df3, on=['subject_id', 'date'], how='inner')
wHr_df3.head(2)

,subject_id,date,hr_afternoon_mean,hr_afternoon_std,hr_afternoon_max,hr_afternoon_min,hr_afternoon_above_100_ratio,hr_evening_mean,hr_evening_std,hr_evening_max,hr_evening_min,hr_evening_above_100_ratio,hr_morning_mean,hr_morning_std,hr_morning_max,hr_morning_min,hr_morning_above_100_ratio,day_night_hr_diff,normal_hr_ratio,hr_stability
0,id01,2024-06-26,80.533300,12.636566,142.0,59.0,0.077268,82.476838,10.293221,124.0,59.0,0.055503,NaN,NaN,NaN,NaN,NaN,-1.943538,0.930558,0.146119
1,id01,2024-06-27,87.405165,12.296994,125.0,63.0,0.179430,83.526477,10.967673,122.0,61.0,0.082859,72.343956,10.333083,130.0,53.0,0.023445,-7.571946,0.925296,0.158047


: 